In [ ]:
!pip install torchtext==0.16.2
!pip install portalocker==2.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import math
from tqdm import tqdm

# **Tạo class quản lý dataset**

In [ ]:
# Tạo class quản lý dataset
class TranslationDataset(Dataset):
    def __init__(self, csv_file, src_lang, tgt_lang):
        # Khởi tạo một list rỗng lưu các cặp src, tgt
        self.data = []
        df = pd.read_csv(csv_file)[:20000] # Lấy 20_000 samples
        for _, row in df.iterrows():
            # Lấy các câu từ src_lang và tgt_lang
            src, tgt = row[src_lang], row[tgt_lang]
            # Thêm cặp src, tgt dưới dạng tuple
            self.data.append((src, tgt))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src, tgt = self.data[idx]
        return src, tgt

In [ ]:
# Khởi tạo tokenize (chung)
tokenizer = get_tokenizer("basic_english")
def tokenize(text):
    return tokenizer(text)

# **Tạo dataset và bộ từ điển riêng cho**

In [ ]:
# Khởi tạo dataset
dataset = TranslationDataset("/content/drive/MyDrive/ML_DL_datasets/en_fr_dataset.csv", src_lang="English words/sentences", tgt_lang="French words/sentences")

In [ ]:
# Tạo hai bộ từ điển (1 cho src, 1 cho tgt)
tokenizer_src = get_tokenizer('basic_english')
tokenizer_tgt = get_tokenizer('basic_english')

In [ ]:
def yield_tokens(data_iter, lang):
    for src, tgt in data_iter:
        if lang == 'src':
            yield tokenizer_src(src)
        else:
            yield tokenizer_tgt(tgt)

In [ ]:
src_vocab = build_vocab_from_iterator(yield_tokens(dataset, 'src'), specials=['<pad>', '<sos>', '<eos>', '<unk>'])
tgt_vocab = build_vocab_from_iterator(yield_tokens(dataset, 'tgt'), specials=['<pad>', '<sos>', '<eos>', '<unk>'])
src_vocab.set_default_index(src_vocab['<unk>'])
tgt_vocab.set_default_index(tgt_vocab['<unk>'])

In [ ]:
src_vocab['<sos>']

1

In [ ]:
src_vocab['<eos>']

2

In [ ]:
src_vocab['<pad>']

0

In [ ]:
src_vocab['<unk>']

3

In [ ]:
# Hàm collate function
# Giúp Pytorch xử lý và biến đổi data đầu vào thành các batch => định dạng phù hợp để huấn luyện mô hình
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src, tgt in batch:
        # Token hóa src (dùng tokenizer)
        # Map từng token thành index từ src_vocab
        # Thêm '<sos>' và '<eos>'
        src_batch.append([src_vocab['<sos>']] + [src_vocab[t] for t in tokenizer_src(src)] + [src_vocab['<eos>']])
        tgt_batch.append([tgt_vocab['<sos>']] + [tgt_vocab[t] for t in tokenizer_tgt(tgt)] + [tgt_vocab['<eos>']])

    # pad các tensor để đảm bảo các câu có độ dài bằng nhau
    src_batch = torch.nn.utils.rnn.pad_sequence([torch.tensor(s) for s in src_batch], padding_value=src_vocab['<pad>'])
    tgt_batch = torch.nn.utils.rnn.pad_sequence([torch.tensor(t) for t in tgt_batch], padding_value=tgt_vocab['<pad>'])

    return src_batch.transpose(0, 1).contiguous(), tgt_batch.transpose(0, 1).contiguous()

In [ ]:
# Tạo Dataloader
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# **Build model**

In [ ]:
# 1. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [ ]:
# 2. Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        # Đây là các ma trận parameters sẽ được đưa vào huấn luyện => giúp model học cách biến đổi input

        # Layer project output sau khi input đã được tổng hợp từ các head (d_head)
        self.W_o = nn.Linear(d_model, d_model, bias=False)


    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        scores = scores.masked_fill(mask == 0, -1e9)
        attn = torch.softmax(scores, dim=-1)
        output = torch.matmul(attn, V)
        return output, attn

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        Q = self.W_q(Q).reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(K).reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(V).reshape(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Attention
        output, attn = self.scaled_dot_product_attention(Q, K, V, mask)

        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(output)
        return output, attn

In [ ]:
# 3. Feed Forward Neural Netwwork
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
# 4. Encoder
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Self-Attention
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        # Feed Forward
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
# 5 Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        # Có 2 attention
        # 1. self_attn giúp model học được mối liên hệ giữa các token với nhau trong câu target
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # 2. cross_attn giúp cho model tham chiếu tới phần src => cross_attn liên kết src với tgt
        # Không có cross_attn => decoder không biết dịch vì không có tham chiếu đến src
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        attn_output, _ = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        # Cross attention => sinh ra target sequence kết hợp tham chiếu đến src thông qua cross_attn
        attn_output, _ = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        # Feed Forward
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
# 6. Transformer Model
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, num_layers=6, d_ff=2048, max_seq_len=100, dropout=0.1):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )

        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)


    # generate_mask
    # che đi các vị trí <pad> => những vị trí không có ý nghĩa
    # Che đi các từ phía sau, chỉ cho phép model dự đoán từ tiếp theo dựa trên các từ phía trước
    # => Quan trọng để model không "gian lận" trong quá trình training
    def generate_mask(self, src, tgt):
        # src != 0 (không phải <pad>) => lấy
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)

        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        # Kết hợp padding mask với nopeak mask
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output


# **Test Model**

In [ ]:
# Khởi tạo mô hình
model = Transformer(len(src_vocab), len(tgt_vocab), d_model=512, num_heads=8, num_layers=6, d_ff=2048)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab['<pad>'])

In [ ]:
# Huấn luyện với tqdm
model.train()
for epoch in range(10):
    total_loss = 0.0
    num_batches = 0
    # Thêm tqdm cho vòng lặp batch
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch}', leave=True)
    for src, tgt in progress_bar:
        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])
        loss = criterion(output.reshape(-1, len(tgt_vocab)), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_batches += 1
        # Cập nhật thanh tiến độ với loss hiện tại
        progress_bar.set_postfix({'batch_loss': loss.item()})
    avg_loss = total_loss / num_batches
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')

Epoch 0: 100%|██████████| 625/625 [03:51<00:00,  2.70it/s, batch_loss=2.77]


Epoch 0, Average Loss: 3.0647


Epoch 1: 100%|██████████| 625/625 [03:52<00:00,  2.69it/s, batch_loss=1.77]


Epoch 1, Average Loss: 1.9192


Epoch 2: 100%|██████████| 625/625 [03:52<00:00,  2.69it/s, batch_loss=1.04]


...

Epoch 6: 100%|██████████| 625/625 [03:53<00:00,  2.68it/s, batch_loss=0.665]


Epoch 6, Average Loss: 0.5658


Epoch 7: 100%|██████████| 625/625 [03:53<00:00,  2.68it/s, batch_loss=0.542]


Epoch 7, Average Loss: 0.4719


Epoch 8:  52%|█████▏    | 325/625 [02:01<01:46,  2.82it/s, batch_loss=0.427]

In [ ]:
def translate_sentence(sentence, model, src_vocab, tgt_vocab, max_length=20):
    model.eval()
    with torch.no_grad():
        # Token hóa câu input
        tokens = [src_vocab['<sos>']]
        for t in tokenizer(sentence):
            tokens.append(src_vocab[t] if t in src_vocab.get_itos() else src_vocab['<unk>'])
        tokens.append(src_vocab['<eos>'])
        src_tensor = torch.LongTensor(tokens).unsqueeze(0)

        # Khởi tạo tensor đầu ra với <sos>
        # Bắt đầu sinh câu dịch bằng <sos> — giống như decoder trong training
        tgt_tokens = [tgt_vocab['<sos>']]
        # Tối đa sinh max_length từ (20) → tránh vòng lặp vô hạn
        for _ in range(max_length):
            # Chuyển list token đã sinh → tensor, thêm batch dim → (1, current_len)
            tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0)
            # Use the full tgt_tensor for the first prediction
            # → Lần đầu: chỉ có <sos> → đưa nguyên tgt_tensor
            # → Từ lần thứ 2: đưa tgt_tensor[:, :-1] (loại bỏ token cuối cùng vừa sinh)
            # → vì model được train với teacher forcing (input là ground truth trừ token cuối)
            output = model(src_tensor, tgt_tensor if len(tgt_tokens) == 1 else tgt_tensor[:, :-1])
            # output: shape (1, current_len, vocab_size)
            # argmax(dim=-1): lấy token có xác suất cao nhất mỗi vị trí
            # [:, -1]: chỉ lấy dự đoán ở vị trí cuối cùng (vừa sinh)
            # .item(): chuyển tensor 1 phần tử → số nguyên Python
            pred_token = output.argmax(dim=-1)[:, -1].item()
            # Thêm token vừa dự đoán vào chuỗi đang sinh
            tgt_tokens.append(pred_token)
            # Dừng khi pred_token là <eos> => kết thúc
            if pred_token == tgt_vocab['<eos>']:
                break

        # Giải mã thành văn bản
        # Bỏ <sos> ở đầu
        # Loại bỏ <pad> và <eos>
        # → danh sách từ theo index → lấy từ tương ứng
        # Ghép các từ bằng dấu cách
        translation = [tgt_vocab.get_itos()[i] for i in tgt_tokens[1:] if i not in [tgt_vocab['<pad>'], tgt_vocab['<eos>']]]
        return ' '.join(translation)

In [ ]:
# Test với input tùy chỉnh
test_sentences = [
    "Hello world",
    "How are you?",
    "My name is Marie",
    "What time is it?"
]

In [ ]:
for sentence in test_sentences:
    translated = translate_sentence(sentence, model, src_vocab, tgt_vocab)
    print(f"English: {sentence}")
    print(f"French: {translated}")
    print("---")